In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    RidgeClassifier,
    SGDClassifier,
)
from sklearn.svm import SVC
from sklearn import tree

In [2]:
df_train = pd.read_csv('../Datasets/loan-prediction/train.csv')
df_train = df_train.drop('Loan_ID', axis=1)
df_train['label'] = 'train'
df_test = pd.read_csv('../Datasets/loan-prediction/test.csv')
df_test = df_test.drop('Loan_ID', axis=1)
df_test['label'] = 'test'
concat_df = pd.concat([df_train , df_test])
cols = set(df_train.columns.tolist())
cols = cols.intersection(df_test.columns.tolist())
cols = list(cols)
df = pd.get_dummies(concat_df)

In [3]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y,label_test,label_train
0,5849,0.0,NaN,360.0,1.0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,0,1
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,0,1,0,0,1,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,1,1,0,0,0,1,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1


In [4]:
train = df[df['label_train'] == 1]
train = train.drop('label_train', axis=1)
train = train.drop('label_test', axis=1)
train = train.drop('Loan_Status_N', axis=1)
train = train.drop('Loan_Status_Y', axis=1)
train['Loan_Status'] = df_train['Loan_Status']
train = train.dropna()
train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,0,1,0,1,0,1,0,0,N
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,0,1,0,0,1,0,0,1,Y
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,Y
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,Y
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,1,Y


In [5]:
test = df[df['label_test'] == 1]
test = test.drop('label_test', axis=1)
test = test.drop('label_train', axis=1)
test = test.drop('Loan_Status_Y', axis=1)
test = test.drop('Loan_Status_N', axis=1)
test = test.fillna(0)
test.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5720,0.0,110.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
1,3076,1500.0,126.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
2,5000,1800.0,208.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
3,2340,2546.0,100.0,360.0,0.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
4,3276,0.0,78.0,360.0,1.0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1


In [6]:
print(len(df_train), len(df_test), len(train), len(test))

614 367 529 367


In [7]:
train_train, train_test = train_test_split(
    train,
    test_size=.3,
    random_state=1,
)

In [8]:
X_train = train_train.drop('Loan_Status', axis=1)
Y_train = train_train['Loan_Status']
X_test = train_test.drop('Loan_Status', axis=1)
Y_test = train_test['Loan_Status']

In [9]:
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
332,2833,0.0,71.0,360.0,1.0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1
399,1500,1800.0,103.0,360.0,0.0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0
256,6045,0.0,115.0,360.0,0.0,0,1,1,0,1,0,0,0,0,1,1,0,1,0,0
116,3167,2283.0,154.0,360.0,1.0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0
148,10000,1666.0,225.0,360.0,1.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0


In [10]:
X_test.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
41,1800,1213.0,47.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
450,2309,1255.0,125.0,360.0,0.0,0,1,0,1,0,0,1,0,0,1,1,0,1,0,0
286,2600,1717.0,99.0,300.0,1.0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,0
124,4300,2014.0,194.0,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,1,0,0
38,4166,7210.0,184.0,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1


In [11]:
Y_train.head()

332    Y
399    N
256    N
116    Y
148    N
Name: Loan_Status, dtype: object

In [12]:
decision_tree = tree.DecisionTreeClassifier(random_state=1)

In [13]:
svc = SVC(
    kernel='poly',
)

In [14]:
sgd = SGDClassifier(
    loss='hinge',
    penalty='l2',
    learning_rate='optimal',
    random_state=1,
    early_stopping=True,
)

In [15]:
gboost = ensemble.GradientBoostingClassifier(
    learning_rate=.01,
    random_state=1,
)

In [16]:
adaboost = ensemble.AdaBoostClassifier(
    #learning_rate=.01,
    #base_estimator=decision_tree,
    #algorithm='SAMME',
    random_state=1,
)

In [17]:
rf = ensemble.RandomForestClassifier(
    #criterion='gini',
)

In [18]:
bagging = ensemble.BaggingClassifier(
    base_estimator=gboost,
    n_estimators=10,
    random_state=1,
)

In [19]:
model = bagging.fit(X_train, Y_train)

In [20]:
model.score(X_test, Y_test)

0.8427672955974843

In [21]:
model = rf.fit(X_train, Y_train)

In [22]:
model.score(X_test, Y_test)

0.8050314465408805

In [23]:
model = adaboost.fit(X_train, Y_train)

In [24]:
model.score(X_test, Y_test)

0.7421383647798742

In [25]:
model = gboost.fit(X_train, Y_train)

In [26]:
model.score(X_test, Y_test)

0.8364779874213837

In [27]:
predicted = model.predict(test)